In [ ]:
%load_ext autoreload
%autoreload 2

# Dataset test

In [ ]:
from PrimeCNNv3.imports import *
from PrimeCNNv3.utils.data.dataset import *

In [ ]:
#hide



SEED = 0

np.random.seed(SEED)
random.seed(SEED)

train_filepath = Path('../../../CovidX/data/train_COVIDx5.txt')

with open(train_filepath,'r') as file:
    cxr_list = np.array([line.rstrip('\n') for line in file])



In [ ]:
train_data = get_dataset(train_filepath, seed=0)

In [ ]:
#hide
train_lst, valid_lst = get_train_val_split(train_data, valid_pct=0.2, seed = 0)

In [ ]:
#hide
#stats

train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
valid_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

#for train ds
for element in train_lst:
    pathlogy = element.split()[-2]
    train_count[pathlogy] += 1

#for valid ds
for element in valid_lst:
    pathlogy = element.split()[-2]
    valid_count[pathlogy] += 1

In [ ]:
#hide
print(train_count)
print(valid_count)

{'normal': 6427, 'pneumonia': 4332, 'COVID-19': 408}
{'normal': 1539, 'pneumonia': 1143, 'COVID-19': 109}


In [ ]:
train_ds = CovidXDataset(Path('../../../CovidX/data/train'), train_data, seed = 42)

In [ ]:
train_ds.show_images(10, nrows=3, rand=False)

In [ ]:
train_ds.show_distribution()

# Learner test

In [ ]:
#hide
import os
from torchvision import datasets, models, transforms
from PrimeCNNv3.utils.data.dataloaders import DataLoaders
from PrimeCNNv3.utils.data.dataset import Dataset
from PrimeCNNv3.callbacks import *
from PrimeCNNv3.metric import *
from PrimeCNNv3.learner import *

In [ ]:
#hide
random_seed(0)
model = models.resnet18(pretrained = True)
model.fc = nn.Linear(512, out_features=10, bias=True)
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
dataset1 = datasets.CIFAR10('../data', train=True, download=True,
                       transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1,batch_size = 64)
                                       
valid_loader = torch.utils.data.DataLoader(testset, batch_size = 64)
dls = DataLoaders(train_loader, valid_loader, device='cuda') 

cbs = [SetupLearnerCB(),Recorder(),ShowStats()]

learn = Learner(model,dls, metric = [accuracy, precision, recall, f1score], 
                cbs = cbs,loss_func=nn.CrossEntropyLoss(), 
                opt_func=torch.optim.AdamW )

In [ ]:
#hide
random_seed(0)
#learn.freeze()
learn.lr_finder()

In [ ]:
random_seed(0)
learn.fit(5,1e-03,wd = 1e-04)

In [ ]:
preds, target = learn.cbs[1].accumetric.get_target_preds_np()

In [ ]:
classes = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plot_confusion_matrix(preds = preds, target=target, labels=classes)